# 01. 데이터 개요

### 1. 데이터셋 설명
> 이번 데이터셋은 **마케팅 캠페인의 효과를 검증하기 위해 A/B 테스트를 진행**한다. <br>
> A/B 테스트는 **무작위 실험 기법**으로, 웹 페이지·배너·광고 등 *서로 다른 버전을 사용자 집단에 동시에 노출하여 성과가 높은 최적의 버전을 선택*하는 방식이다.
>
> 마케팅 회사들은 두 가지 질문에 답하고자 한다.
> 1. 캠페인이 성공할 수 있는가?
> 2. 성공했다면, 그 성공의 얼마나 많은 부분이 **광고 노출**에 기인하는가?
>
> 이를 위해 대다수 사용자(실험군)는 광고(ad)에 노출되고, 소수 사용자(대조군)는 공익광고(psa) 또는 아무것도 보지 않는다. <br>
> 데이터셋은 이러한 실험 설계를 기반으로 하며, **광고가 실제로 전환(구매 등)에 영향을 미쳤는지** 검증하는 데 목적이 있다.

### 2. 분석 목표
- 실험군(`ad`)과 대조군(`psa`) 간 전환율 차이 검증  
- 전환에 영향을 미치는 요인 탐색  
<font style="color:#00198f">ex) 광고 노출량(`total ads`)이 전환에 미치는 영향 분석, 요일(`most ads day`), 시간대(`most ads hour`)에 따른 전환 패턴 탐색</font>

### 3. 분석 프로세스
1. 데이터 개요 → 2. 탐색적 데이터 분석 (EDA) → 3. 통계적 검정 (A/B Test) → 4. 시각화 및 대시보드 제작 (Tableau) → 5. 인사이트 요약 및 의사결정 제시

### 4. 변수 설명
- **Unnamed: 0**: 단순 인덱스 값 (불필요; 제거 대상)
- **user id**: 사용자 <font style="color:#00198f">(ex: 1069124)</font>
- **test group**: 실험군 / 대조군 <font style="color:#00198f">(`ad` / `psa`)</font>
- **converted**: 전환 여부 <font style="color:#00198f">(`True` / `False`)</font>
- **total ads**: 사용자가 본 총 광고 수 <font style="color:#00198f">(ex: 130)</font>
- **most ads day**: 가장 많은 광고가 노출된 요일 <font style="color:#00198f">(Monday-Sunday)</font>
- **most ads hour**: 가장 많은 광고가 노출된 시간대 <font style="color:#00198f">(0-23)</font>

### 5. 데이터 출처
> Kaggle : https://www.kaggle.com/datasets/faviovaz/marketing-ab-testing

<hr>

## 1. 데이터 불러오기

In [1]:
import pandas as pd

df = pd.read_csv('../dataset/marketing_AB.csv')
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


## 2. 데이터 확인

In [2]:
# 데이터 크기 (행, 열)
df.shape

(588101, 7)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


In [4]:
# 기초 통계 (수치형)
df.describe()

,Unnamed: 0,user id,total ads,most ads hour
count,588101.000000,5.881010e+05,588101.000000,588101.000000
mean,294050.000000,1.310692e+06,24.820876,14.469061
std,169770.279667,2.022260e+05,43.715181,4.834634
min,0.000000,9.000000e+05,1.000000,0.000000
25%,147025.000000,1.143190e+06,4.000000,11.000000
50%,294050.000000,1.313725e+06,13.000000,14.000000
75%,441075.000000,1.484088e+06,27.000000,18.000000
max,588100.000000,1.654483e+06,2065.000000,23.000000


## 3. 범주형 변수 탐색

In [5]:
# object 타입 컬럼 확인
cols = df.select_dtypes(include='object').columns

# 각 범주형 변수별 분포
for col in cols:
    print(f"\n--- {col} ---")
    print(df[col].value_counts())


--- test group ---
test group
ad     564577
psa     23524
Name: count, dtype: int64

--- most ads day ---
most ads day
Friday       92608
Monday       87073
Sunday       85391
Thursday     82982
Saturday     81660
Wednesday    80908
Tuesday      77479
Name: count, dtype: int64


## 4. 결측치 확인

In [6]:
# 결측치 개수 확인
print(df.isnull().sum())

Unnamed: 0       0
user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64


## 5. 중복 여부 확인

In [20]:
# user id 기준 중복 행 출력
df[df['user id'].duplicated()]

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour


## 6. 요약
- 데이터는 총 588,101행 x 7열로 구성
- 불필요한 컬럼(`Unnamed: 0`)은 향후 분석 시 제거
- 실험군(`ad`)이 데이터의 대다수, 대조군(`psa`)은 소수
- **결측치 X, 중복 X**